<a href="https://colab.research.google.com/github/JISOO0213/finger_nimi_project/blob/main/finger_nimi_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 255 kB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    image = cv2.flip(image, 1)
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    # image.flags.writeable = False
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True

    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        joints = np.zeros((5, 4, 3))
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        # finger split
        for i, lm in enumerate(hand_landmarks.landmark):
          if i == 0:
            pass
          if i % 4 != 0:
            joints[i // 4][i % 4 - 1] = (lm.x, lm.y, lm.z)
          else:
            joints[i // 4 - 1][3] = (lm.x, lm.y, lm.z)

        # finger joint
        thum = [False, False]
        stretch = [False] * 5

        # finger up or down
        for i in range(1, 5):
          if (joints[i][1][1] - joints[i][0][1]) * (joints[i][3][1] - joints[i][2][1]) > 0:
            stretch[i] = True

        if (joints[0][1][0] - joints[0][0][0]) * (joints[0][3][0] - joints[0][2][0]) > 0:
          stretch[0] = True

        if (joints[0][1][1] - joints[0][0][1]) * (joints[0][3][1] - joints[0][2][1]) > 0:
          thum[0] = True
          if joints[0][1][1] - joints[0][0][1] > 0:
            thum[1] = True

        # finger pose 
        if stretch[0] and stretch[1] and stretch[4] and not (stretch[2] and stretch[3]):
          cv2.putText(image, 'nico-nico-ni >_<', (50, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 5, cv2.LINE_AA)
        elif stretch[3] and thum[0] and not (stretch[0] and stretch[1] and stretch[2] and stretch[4]):
          cv2.putText(image, 'i love you too much', (50, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 5, cv2.LINE_AA,)
        elif thum[0] and not thum[1] and not (stretch[1] and stretch[2] and stretch[3] and stretch[4]):
          cv2.putText(image, 'thum-up', (50, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 5, cv2.LINE_AA)
        elif thum[0] and thum[1] and not (stretch[1] and stretch[2] and stretch[3] and stretch[4]):
          cv2.putText(image, 'YOU DIE', (50, 200), cv2.FONT_HERSHEY_PLAIN, 7, (0, 0, 255), 5, cv2.LINE_AA)

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()